In [1]:
!pip install fastai==2.7.9
!pip install fastkaggle==0.0.7
!pip install fasterai==0.1.11
!pip install ipython-autotime==0.3.1
!pip install torch==1.12.1
%load_ext autotime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.5/225.5 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 738.0 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 47.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 53.2 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.0.0
    Uninstalling torch-2.0.0:
      Successfully uninstalled torch-2.0.0
  Attempting uninstall: fastai
    Found existing installation: fastai 2.7.12
    Uninstalling fastai-2.7.12:
      Successfully uninstalled fastai-2.7.12
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchdata 0.6.0 requires torch==2.0.

In [2]:
from fastai.vision.all import *
from fastai.callback.all import *
from fastai.data.all import *
from fastai.imports import *
from fastai.torch_core import *
from fastai.learner import *
from fastai.fp16_utils import *
from fastai.callback.fp16 import *
import fastai.optimizer as optim


from fasterai.sparse.all import *
from fasterai.core.criteria import *
from fasterai.regularize.all import *

from fastkaggle import *

import torch
import torch.nn as nn
import torch.nn.functional as F

set_seed(42)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/opt/conda/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN3c104cuda20CUDACachingAllocator9allocatorE'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


time: 3.8 s (started: 2023-08-09 09:23:46 +00:00)


In [3]:
comp = 'paddy-diseases/paddy-diseases'

path = setup_comp(comp, install='"fastcore>=1.4.5" "fastai>=2.7.1" "timm>=0.6.2.dev0"')

time: 12.7 s (started: 2023-08-09 09:23:50 +00:00)


In [4]:
dls = ImageDataLoaders.from_folder(path/"train", 
                                   valid_pct=0.25, 
                                   item_tfms=Resize(256), 
                                   batch_tfms=
                                   aug_transforms(   size=64, 
                                                                min_scale=0.75,
                                                                max_rotate=10., 
                                                                min_zoom=0.8, 
                                                                max_zoom=1.2, 
                                                            
                                                            ),
                                   bs=32)

time: 5.44 s (started: 2023-08-09 09:24:03 +00:00)


In [5]:
model = vgg16_bn(num_classes=10).cuda()

time: 2.13 s (started: 2023-08-09 09:24:08 +00:00)


In [6]:
cbs = [     
#         GradientAccumulation(32), 
#         SparsifyCallback(sparsity=25, 
#                          granularity='filter', 
#                          context='local', 
#                          criteria=large_final, 
#                          schedule=one_cycle),
        ]

time: 493 µs (started: 2023-08-09 09:24:11 +00:00)


In [7]:
from fastai.learner import Learner

class CustomLearner(Learner):
    optimizer_memory_usage = 0
    def _step(self):
        before_memory = torch.cuda.max_memory_allocated(device=torch.device("cuda"))
        super()._step()
        after_memory = torch.cuda.max_memory_allocated(device=torch.device("cuda"))
      
        # calculate the memory usage during the optimizer step
        self.optimizer_memory_usage += after_memory - before_memory
    
        print(f"Optimizer memory usage: {self.optimizer_memory_usage} Byte")


time: 853 µs (started: 2023-08-09 09:24:11 +00:00)


In [8]:
learn = CustomLearner(dls, 
                model = model, 
                metrics=[accuracy,
                         error_rate,
                        ],
               ).to_fp16()
learn.unfreeze()
learn.recorder.train_metrics = True
learn.path = Path("/kaggle/working/")



time: 7.12 ms (started: 2023-08-09 09:24:11 +00:00)


In [9]:
learn.fit_one_cycle(1, 3e-5, cbs=cbs)

epoch,train_loss,train_accuracy,train_error_rate,valid_loss,valid_accuracy,valid_error_rate,time
0,2.157320,0.214423,0.785577,1.932548,0.320673,0.679327,01:13


Optimizer memory usage: 930691072 Byte
Optimizer memory usage: 930691072 Byte
Optimizer memory usage: 930691072 Byte
Optimizer memory usage: 930691072 Byte
Optimizer memory usage: 930691072 Byte
Optimizer memory usage: 930691072 Byte
Optimizer memory usage: 930691072 Byte
Optimizer memory usage: 930691072 Byte
Optimizer memory usage: 930691072 Byte
Optimizer memory usage: 930691072 Byte
Optimizer memory usage: 930691072 Byte
Optimizer memory usage: 930691072 Byte
Optimizer memory usage: 930691072 Byte
Optimizer memory usage: 930691072 Byte
Optimizer memory usage: 930691072 Byte
Optimizer memory usage: 930691072 Byte
Optimizer memory usage: 930691072 Byte
Optimizer memory usage: 930691072 Byte
Optimizer memory usage: 930691072 Byte
Optimizer memory usage: 930691072 Byte
Optimizer memory usage: 930691072 Byte
Optimizer memory usage: 930691072 Byte
Optimizer memory usage: 930691072 Byte
Optimizer memory usage: 930691072 Byte
Optimizer memory usage: 930691072 Byte
Optimizer memory usage: 9